<a href="https://colab.research.google.com/github/prachiacharya01/SneakPeek/blob/main/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install implicit
!pip install gmplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 56.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 164 kB 7.0 MB/s 


## Importing Dependencies and Datasets

In [ ]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

import gmplot 

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


In [ ]:
venues_df = pd.read_csv('/content/drive/MyDrive/ALS_dataset/venues.csv')
spot_df = pd.read_csv('/content/drive/MyDrive/ALS_dataset/spot_df')

In [ ]:
venues_df.drop(['Unnamed: 0'], axis=1, inplace=True)
spot_df.drop(['Unnamed: 0'], axis=1, inplace=True)
spot_df.isna().sum()
venues_df.isna().sum()
venues_df = venues_df.drop_duplicates()
spot_df = spot_df.drop_duplicates()

In [ ]:
spot_df['eventType'].value_counts()

LIKE               1610968
BOOKMARK           1610908
VIEW               1610774
COMMENT CREATED    1610058
Name: eventType, dtype: int64

# Feature Engineering

Merging above two datasets on the basis of spotid

In [ ]:
merged_df = pd.merge(venues_df,spot_df, on='spotid',how='inner')

In [ ]:
merged_df = merged_df.drop_duplicates()

In [ ]:
merged_df.drop(['geo-coordinates'], axis=1)
merged_df['spotid'] = merged_df['spotid'].astype('category')
merged_df['userid'] = merged_df['userid'].astype('category')

In [ ]:
merged_df['s_id'] = merged_df['spotid'].cat.codes
merged_df['u_id'] = merged_df['userid'].cat.codes
merged_df

,spotid,spotname,geo-coordinates,userid,timestamp,latitude,longitude,eventType,s_id,u_id
0,1391604,Conference House Park,POINT(-74.2523431778 40.5017589436),4929,2010-07-05T16:48:52Z,40.501759,-74.252343,BOOKMARK,15105,815
1,1391611,Almer G. Russell Pavilion,POINT(-74.2542636395 40.5022647413),4929,2010-07-05T16:50:26Z,40.502265,-74.254264,COMMENT CREATED,15106,815
2,1391499,Biddle House,POINT(-74.2541456223 40.5054829933),4929,2010-07-05T16:29:01Z,40.505483,-74.254146,LIKE,15104,815
3,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19026,2010-02-22T01:59:37Z,40.503929,-74.263209,COMMENT CREATED,5899,1812
4,403372,Seabra's Armory,POINT(-74.2632088065 40.5039289517),19346,2010-02-27T07:48:40Z,40.503929,-74.263209,VIEW,5899,1836
...,...,...,...,...,...,...,...,...,...,...
114705,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,2010-04-14T17:18:34Z,40.898167,-73.854614,COMMENT CREATED,11389,6619
114706,891282,Peking House,POINT(-73.8546144962 40.898167491),166694,2010-04-07T17:41:28Z,40.898167,-73.854614,BOOKMARK,11389,6619
114707,397723,Larchmont Club,POINT(-73.745129 40.9244924667),378,2010-01-16T23:15:41Z,40.924492,-73.745129,COMMENT CREATED,5822,89
114708,1224978,Zitoune,POINT(-73.75237983 40.94017511),18055,2010-08-28T19:29:19Z,40.940175,-73.752380,BOOKMARK,14099,1773


Strength evaluation on the basis of selected event

In [ ]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 3.0, 
   'COMMENT CREATED': 4.0,  
}

merged_df['eventStrength'] = merged_df['eventType'].apply(lambda x:event_type_strength[x])

In [ ]:
merged_df

# Matrix Generation

In [ ]:
# naming convention: sparse_row_column
sparse_spot_person = sparse.csr_matrix((merged_df['eventStrength'].astype(float), (merged_df['s_id'], merged_df['u_id'])))
sparse_person_spot = sparse.csr_matrix((merged_df['eventStrength'].astype(float), (merged_df['u_id'], merged_df['s_id'])))

# Model

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


In [ ]:
alpha = 15
data = (sparse_spot_person * alpha).astype('double')
# Fit the model
model.fit(data)

  0%|          | 0/50 [00:00<?, ?it/s]

function of recommendation

In [ ]:
def recommend(person_id, sparse_person_content, person_vecs, content_vecs, num_contents=10):
    person_interactions = sparse_person_content[person_id,:].toarray()
    person_interactions = person_interactions.reshape(-1) + 1
    person_interactions[person_interactions > 1] = 0
# [0,0+1]
    rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    person_interactions = person_interactions[:rec_vector_scaled.shape[0]]

    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = person_interactions * rec_vector_scaled

    # Sort the indices of the content into order of best recommendations
    spot_idx = np.argsort(recommend_vector)[::-1][:num_contents]

    spot_names = []
    scores = []
    latitude = []
    longitude = []
    for idx in spot_idx:
      spot_names.append(merged_df['spotname'][merged_df['s_id']==idx].iloc[0])
      latitude.append(merged_df['latitude'][merged_df['s_id']==idx].iloc[0])
      longitude.append(merged_df['longitude'][merged_df['s_id']==idx].iloc[0])
      scores.append(recommend_vector[idx])
    
    data = {
        'Spotnames': spot_names, 
        'Score': scores,
        'Latitude':latitude,
        'Longitude':longitude
        }
    recommendations = pd.DataFrame(data)

    return recommendations

recommenations based on user's previous history

In [ ]:
person_vecs = sparse.csr_matrix(model.user_factors)
content_vecs = sparse.csr_matrix(model.item_factors)

person_id = 50
# u_lat = 40.691247
# u_long = -74.179516
# merged_df = merged_df[(merged_df['latitude']> u_lat-1 ) & 
#                       (merged_df['latitude']< u_lat+1) &
#                       (merged_df['longitude']>u_long -1) &
#                       (merged_df['longitude']<u_long + 1)]

recommendations = recommend(person_id, sparse_person_spot, person_vecs, content_vecs)
print(recommendations)

                   Spotnames     Score   Latitude  Longitude
0          Terminal 8 at JFK  1.000000  40.648899 -73.792570
1        Caputo's Fine Foods  0.982100  40.678125 -73.997818
2                92Y Tribeca  0.972828  40.722917 -74.007393
3                 Mrs. Kim's  0.958861  40.730838 -73.957344
4            Pace University  0.949848  40.710461 -74.004207
5            Babe Ruth Plaza  0.935799  40.828532 -73.927345
6         NYC Fried Chicken   0.929608  40.756337 -73.992926
7        MakerBot Industries  0.929308  40.683374 -73.982018
8  Skull Crusher Mountain NJ  0.926289  40.732012 -74.070790
9                      Giano  0.917503  40.726223 -73.984272


user's previous history 

In [ ]:
# user_id == 50
merged_df.loc[merged_df['u_id'] == 50].sort_values(by=['eventStrength'], ascending=False)[['spotname', 'u_id', 'eventStrength','latitude','longitude']].head(10)

,spotname,u_id,eventStrength,latitude,longitude
3638,EWR Newark Liberty International,50,2.0,40.691247,-74.179516
47273,The Daily Show,50,2.0,40.766819,-73.993867
3639,EWR Newark Liberty International,50,1.0,40.691247,-74.179516
29841,Starbucks,50,1.0,40.749671,-73.994035


# Ploting points on map

In [ ]:
!pip install chart-studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.3 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=51850d9f7d396a1c4eff08581c86f871f3910195323b332de887eab21e4d141f
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [ ]:
from chart_studio import plotly as py
import chart_studio
import plotly.figure_factory as ff
import plotly.graph_objs as go

chart_studio.tools.set_credentials_file(
    username='prachi2302',
    api_key='qXaUZOLe1LekH11C4gBu'
)

mapbox_access_token = 'pk.eyJ1IjoiYXNjaHJvY2siLCJhIjoiY2p2NnRoeHc2MDkxbTQ0bnR6aTVwZDNsaCJ9.MA76hkxD3rOGgnVCDBVC9w'

df = recommendations

site_lat = df.Latitude
site_lon = df.Longitude
locations_name = df.Spotnames
	
# Generate the data for the map 

data = [
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=3,
            color='rgb(255, 0, 0)',
            opacity=1
        ),
        text=locations_name,
        hoverinfo='text'
    ),
    go.Scattermapbox(
        lat=site_lat,
        lon=site_lon,
        mode='markers',
        marker=dict(
            size=8,
            color='rgb(242, 177, 172)',
            opacity=0.8
        ),
        hoverinfo='none'
    )]


# Generate a layout around Los Angeles, zoomed in so we can see the data points 

layout = go.Layout(
    title='Recommeded Places',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=40.691247,
            lon=-74.179516
        ),
        pitch=0,
        zoom=10,
        style='dark'
    ),
)
	
# Generate the figure using the iplot function 
  
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='Recommendation')